In [26]:
import re

cigar_string = "3M1D3M1D5M2I"
cigar = re.findall(r'(\d+)([MID])', cigar_string)

# Initialize read and reference tensors
read = torch.zeros(len(cigar), dtype=torch.int32)
ref = torch.zeros(len(cigar), dtype=torch.int32)

# Initialize read and reference positions
read_pos = 0
ref_pos = 0

# Update read and reference positions based on CIGAR operations
for i, (length, op_type) in enumerate(cigar):
    # print(i, (length, op_type))
    length = int(length)
    if op_type == 'M':  # Match or mismatch
        read[i] = read_pos + length
        ref[i] = ref_pos + length
        read_pos += length
        ref_pos += length
    elif op_type == 'D':  # Deletion
        read[i] = read_pos
        ref[i] = ref_pos + length
        ref_pos += length
    elif op_type == 'I':  # Insertion
        read[i] = read_pos + length
        ref[i] = ref_pos
        read_pos += length
print("read = ",read)
print("ref = ",ref)

read =  tensor([ 3,  3,  6,  6, 11, 13], dtype=torch.int32)
ref =  tensor([ 3,  4,  7,  8, 13, 13], dtype=torch.int32)


In [27]:
# Initialize tensor with zeros
alignment_tensor = torch.zeros(read[-1], ref[-1], 4, dtype=torch.int32)

# Fill in tensor based on CIGAR operations
for i, (length, op_type) in enumerate(cigar):
    length = int(length)
    if op_type == 'M':  # Match or mismatch
        alignment_tensor[read[i-1]:read[i], ref[i-1]:ref[i], 0] = 1
        print(alignment_tensor)
    elif op_type == 'D':  # Deletion
        alignment_tensor[read[i-1]:read[i], ref[i-1]:ref[i], 1] = 1
    elif op_type == 'I':  # Insertion
        alignment_tensor[read[i-1]:read[i], ref[i-1]:ref[i], 2] = 1
# print(alignment_tensor)

tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
     

In [40]:
import torch

cigar = "3M1D2M1I4M"  # example CIGAR string
reference_sequence = "ACGTACGTAC"  # example reference sequence
read_sequence = "AGTAC-TAG"  # example read sequence
match_symbol = 0  # symbol representing match (M)
insertion_symbol = 1  # symbol representing insertion (I)
deletion_symbol = 2  # symbol representing deletion (D)
#ACGTACGTAC
#A GTAC-TAG
# convert CIGAR string to list of tuples representing each operation
cigar_ops = [(int(cigar[i:i+1]), cigar[i+1:i+2]) for i in range(0, len(cigar), 2)]

# determine length of the resulting tensor
num_ops = len(cigar_ops)
max_length = len(reference_sequence) + sum(op[0] for op in cigar_ops if op[1] == "I") - sum(op[0] for op in cigar_ops if op[1] == "D")
tensor = torch.zeros(num_ops, max_length, dtype=torch.int32)

# fill tensor with appropriate symbols for each operation
read_pos = 0
ref_pos = 0
for i, op in enumerate(cigar_ops):
    length, op_type = op
    if op_type == "M":
        for j in range(length):
            if read_pos < len(read_sequence) and ref_pos < len(reference_sequence) and read_sequence[read_pos] == reference_sequence[ref_pos]:
                tensor[i, j] = match_symbol
            else:
                tensor[i, j] = insertion_symbol
            read_pos += 1
            ref_pos += 1
    elif op_type == "I":
        for j in range(length):
            if read_pos < len(read_sequence):
                tensor[i, j] = insertion_symbol
                read_pos += 1
    elif op_type == "D":
        for j in range(length):
            if ref_pos < len(reference_sequence):
                tensor[i, j] = deletion_symbol
                ref_pos += 1

print(tensor)


tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [43]:
import torch

# define CIGAR strings
cigar1 = "3M1D2M1I4M"
cigar2 = "2M2I1D3M"

# define other variables
reference_sequence = "ACGTACGTAC"
match_symbol = 0  # symbol representing match (M)
insertion_symbol = 1  # symbol representing insertion (I)
deletion_symbol = 2  # symbol representing deletion (D)

# define function to convert CIGAR string to tensor
def cigar_to_tensor(cigar, reference_sequence, match_symbol, insertion_symbol, deletion_symbol):
    # convert CIGAR string to list of tuples representing each operation
    cigar_ops = [(int(cigar[i:i+1]), cigar[i+1:i+2]) for i in range(0, len(cigar), 2)]

    # determine length of the resulting tensor
    num_ops = len(cigar_ops)
    max_length = len(reference_sequence) + sum(op[0] for op in cigar_ops if op[1] == "I") - sum(op[0] for op in cigar_ops if op[1] == "D")
    tensor = torch.zeros(num_ops, max_length, dtype=torch.int32)

    # fill tensor with appropriate symbols for each operation
    read_pos = 0
    ref_pos = 0
    for i, op in enumerate(cigar_ops):
        length, op_type = op
        if op_type == "M":
            for j in range(length):
                if  read_pos < len(read_sequence) and ref_pos < len(reference_sequence) and read_sequence[read_pos] == reference_sequence[ref_pos]:
                    tensor[i, j] = match_symbol
                else:
                    tensor[i, j] = insertion_symbol
                read_pos += 1
                ref_pos += 1
        elif op_type == "I":
            for j in range(length):
                tensor[i, j] = insertion_symbol
                read_pos += 1
        elif op_type == "D":
            for j in range(length):
                tensor[i, j] = deletion_symbol
                ref_pos += 1

    return tensor

# build tensors from CIGAR strings
tensor1 = cigar_to_tensor(cigar1, reference_sequence, match_symbol, insertion_symbol, deletion_symbol)
tensor2 = cigar_to_tensor(cigar2, reference_sequence, match_symbol, insertion_symbol, deletion_symbol)
print(tensor1)
print(tensor2)

# concatenate tensors along 0th dimension
concatenated_tensor = torch.cat([tensor1, tensor2], dim=0)

# print resulting tensor
print(concatenated_tensor)


tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 10 but got size 11 for tensor number 1 in the list.

In [44]:
import re

cigar_string = "10M2I4M1D3M"

insertions = []
deletions = []
matches = []
substitutions = []
cigar_ops = re.findall(r"\d+|[A-Z]", cigar_string)

position = 0

for i in range(0, len(cigar_ops), 2):
    length = int(cigar_ops[i])
    op = cigar_ops[i + 1]

    if op == "M":
        for j in range(position, position + length):
            matches.append(j)
        position += length
    elif op == "I":
        for j in range(position, position + length):
            insertions.append(j)
    elif op == "D":
        for j in range(position, position + length):
            deletions.append(j)
        position += length
    elif op == "X":
        for j in range(position, position + length):
            deletions.append(j)
        position += length
    else:
        raise ValueError("Invalid CIGAR operation: " + op)

print("Insertions:", insertions)
print("Deletions:", deletions)
print("Matches:", matches)

Insertions: [10, 11]
Deletions: [14]
Matches: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17]


In [18]:
import re

cigar_string = "10M2I4M1D3M"

insertions = []
deletions = []
substitutions = []
matches = []

cigar_ops = re.findall(r"\d+|[A-Z]", cigar_string)

position = 0

for i in range(0, len(cigar_ops), 2):
    length = int(cigar_ops[i])
    op = cigar_ops[i + 1]

    if op == "M":
        for j in range(position, position + length):
            matches.append(j)
        position += length
    elif op == "I":
        for j in range(position, position + length):
            insertions.append(j)
        position += length
    elif op == "D":
        for j in range(position, position + length):
            deletions.append(j)
        position += length
    else:
        raise ValueError("Invalid CIGAR operation: " + op)

print("Insertions:", insertions)
print("Deletions:", deletions)
print("Substitutions:", substitutions)
print("Matches:", matches)




Insertions: [11, 12]
Deletions: [17]
Substitutions: []
Matches: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 18, 19, 20]


In [17]:
import re
import torch
def parse_cigar(cigar_string,max_length):
    # Initialize empty lists for insertions, deletions, and matches
    insertions = []
    deletions = []
    matches = []

    # Parse the CIGAR string to extract the operation codes and lengths
    cigar_ops = re.findall(r"\d+|[A-Z]", cigar_string)

    # Loop over the operations to compute the positions of insertions, deletions, and matches
    position = 1
    for i in range(0, len(cigar_ops), 2):
        length = int(cigar_ops[i])
        op = cigar_ops[i + 1]

        if op == "M":
            for j in range(position, position + length):
                matches.append(j)
            position += length
        elif op == "I":
            for j in range(position, position + length):
                insertions.append(j)
            position += length
        elif op == "D":
            for j in range(position, position + length):
                deletions.append(j)
            position += length

    # Pad the shorter lists with zeros
    matches += [0] * (max_length - len(matches))
    insertions += [0] * (max_length - len(insertions))
    deletions += [0] * (max_length - len(deletions))
    whole_cigar = [matches,insertions,deletions]
    return whole_cigar


def cigar_lists(filename,max_length):
    cigar_list = []
    with open(filename, 'r') as file:
        for line in file:
            # Do something with the line
            wc = parse_cigar(line,max_length)
            cigar_list.append(wc)
    tensor = torch.tensor(cigar_list)
    return tensor
# Example CIGAR string
cigar_string = "10M2I4M1D3M"

# Initialize empty lists for insertions, deletions, and matches
insertions = []
deletions = []
matches = []

# Parse the CIGAR string to extract the operation codes and lengths
cigar_ops = re.findall(r"\d+|[A-Z]", cigar_string)

# Loop over the operations to compute the positions of insertions, deletions, and matches
position = 1
for i in range(0, len(cigar_ops), 2):
    length = int(cigar_ops[i])
    op = cigar_ops[i + 1]

    if op == "M":
        for j in range(position, position + length):
            matches.append(j)
        position += length
    elif op == "I":
        for j in range(position, position + length):
            insertions.append(j)
        position += length
    elif op == "D":
        for j in range(position, position + length):
            deletions.append(j)
        position += length

# Pad the shorter lists with zeros
max_length = max(len(matches), len(insertions), len(deletions))
matches += [0] * (max_length - len(matches))
insertions += [0] * (max_length - len(insertions))
deletions += [0] * (max_length - len(deletions))
whole_cigar = [matches,insertions,deletions]
print(whole_cigar)
# Combine the three lists into a single tensor
tensor = torch.tensor(whole_cigar)

# Print the resulting tensor
# print("Tensor:", tensor)


# filename = "test.txt"
# l = cigar_lists(filename,109)
# print(l)



[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 18, 19, 20], [11, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [40]:
cigar_str = "2M1I1D3M"

def cigar_to_symbols_and_positions(cigar):
    symbols = ""
    positions = []
    reference_pos = 0
    cigar_tokens = re.findall(r'(\d+)([MIDNSHP=X])', cigar)
    for token in cigar_tokens:
        length, op = int(token[0]), token[1]
        if op == 'M':
            symbols += 'M' * length
            reference_pos += length
        elif op == 'I':
            symbols += 'I' * length
            reference_pos += length
        elif op == 'D':
            symbols += 'D' * length
            positions.extend(range(reference_pos, reference_pos + length))
            reference_pos += length
        else:
            symbols += op * length
            reference_pos += length
    return symbols


cigar_101 = "2M1D30M1D4M1I63M1D1M"
print(cigar_to_symbols_and_positions(cigar_101))
# print(len(cigar_to_symbols(cigar_101)))

104
